In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [82]:
df = pd.read_csv('train.csv')

In [83]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0


In [84]:
df.columns



Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [85]:
df = pd.get_dummies(df,columns=['gender','ever_married','work_type','Residence_type'])

In [91]:
X = df.drop(['id','smoking_status','stroke','gender_Other','work_type_Never_worked'],axis=1)

In [92]:
df.columns

Index(['id', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level',
       'bmi', 'smoking_status', 'stroke', 'gender_Female', 'gender_Male',
       'gender_Other', 'ever_married_No', 'ever_married_Yes',
       'work_type_Govt_job', 'work_type_Never_worked', 'work_type_Private',
       'work_type_Self-employed', 'work_type_children', 'Residence_type_Rural',
       'Residence_type_Urban'],
      dtype='object')

In [93]:
y = df['stroke']

In [94]:
X.columns

Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'gender_Female', 'gender_Male', 'ever_married_No', 'ever_married_Yes',
       'work_type_Govt_job', 'work_type_Private', 'work_type_Self-employed',
       'work_type_children', 'Residence_type_Rural', 'Residence_type_Urban'],
      dtype='object')

In [95]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.1, random_state =4)


In [96]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform((X))
X_test = sc.transform(X_test)


In [97]:
from keras.utils import np_utils
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255



In [98]:
Y_train = np_utils.to_categorical(y_train, 26)
Y_test = np_utils.to_categorical(y_test, 26)


In [99]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [110]:
ann = Sequential()
ann.add(Dense(units=17, activation='relu'))
ann.add(Dense(units=9, activation='relu'))

ann.add(Dense(units=1, activation='softplus'))


opt = tf.keras.optimizers.Adam()
ann.compile(loss="binary_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [111]:
ann.fit(X_train,y_train,batch_size=32,validation_data=(X_test,y_test),epochs=50)

Epoch 1/50
431/431 [==============================] - 2s 2ms/step - loss: 0.4344 - accuracy: 0.7473 - val_loss: 0.1944 - val_accuracy: 0.9673
Epoch 2/50
431/431 [==============================] - 1s 1ms/step - loss: 0.1834 - accuracy: 0.9577 - val_loss: 0.1732 - val_accuracy: 0.9673
Epoch 3/50
431/431 [==============================] - 1s 1ms/step - loss: 0.1753 - accuracy: 0.9577 - val_loss: 0.1516 - val_accuracy: 0.9673
Epoch 4/50
431/431 [==============================] - 1s 1ms/step - loss: 0.1639 - accuracy: 0.9577 - val_loss: 0.1423 - val_accuracy: 0.9673
Epoch 5/50
431/431 [==============================] - 1s 1ms/step - loss: 0.1502 - accuracy: 0.9577 - val_loss: 0.1517 - val_accuracy: 0.9673
Epoch 6/50
431/431 [==============================] - 1s 2ms/step - loss: 0.1392 - accuracy: 0.9577 - val_loss: 0.1758 - val_accuracy: 0.9673
Epoch 7/50
431/431 [==============================] - 1s 2ms/step - loss: 0.1338 - accuracy: 0.9581 - val_loss: 0.1848 - val_accuracy: 0.9673
Epoch 

In [68]:
ann.evaluate(X_test, y_test)
y_pred = ann.predict(X_test)


48/48 [==============================] - 0s 992us/step


In [69]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [70]:
cm

array([[1481,    0],
       [   0,   50]], dtype=int64)

In [71]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=1000)

In [72]:
clf.score(X_test,y_test)


0.9679947746570868

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
X = pca.fit_transform(X)

In [73]:
df_t = pd.read_csv('test.csv')
df_t = pd.get_dummies(df_t, columns=['gender', 'ever_married', 'work_type', 'Residence_type'])
id = df_t['id']
X_t = df_t.drop(['id','smoking_status'],axis=1)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_t = sc.fit_transform((X_t))
from keras.utils import np_utils
X_t = X_t.astype('float32')

X_t /= 255

y_p = ann.predict(X_t)
print(len(X_t))
print(len(y_p))

319/319 [==============================] - 0s 933us/step
10204
10204


In [76]:
df_t['stroke']=pd.Series([i[0] for i in y_p])
df_t['id']=id
csc = df_t[['id','stroke']]
csc.to_csv('Submission_file_name.csv', index=False , header = 1)


In [131]:
with open('as.csv','w+') as ap:
    for i in y_p:
        ap.write(str(i[0]))
        ap.write('\n')

In [133]:
len(y_p)

10204